<a href="https://colab.research.google.com/github/raagKush/ML_learning_google_collab/blob/main/CLIP_vlm_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install torch torchvision torchaudio
!pip install transformers


In [2]:
!git clone https://github.com/openai/CLIP.git
%cd CLIP
!pip install -r requirements.txt


Cloning into 'CLIP'...
remote: Enumerating objects: 256, done.
remote: Total 256 (delta 0), reused 0 (delta 0), pack-reused 256 (from 1)
Receiving objects: 100% (256/256), 8.93 MiB | 14.17 MiB/s, done.
Resolving deltas: 100% (133/133), done.
/content/CLIP
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 3.8 MB/s eta 0:00:00


In [15]:
import torch
import clip
from PIL import Image
import requests
from io import BytesIO

# Load the CLIP model
device = "cuda" if torch.cuda.is_available() else "cpu"
# Load the RN50 model instead of ViT-B/32
model, preprocess = clip.load("RN50", device)


# Fetch an image from the web
image_path = "/content/cat.jpeg"
img = Image.open(image_path)

# Preprocess the image and encode it
img_input = preprocess(img).unsqueeze(0).to(device)

# Define text descriptions
texts = ["A dog", "A cat", "A person", "A golden retriever","An orange Tabby Cat sitting", "car"]
text_inputs = clip.tokenize(texts).to(device)

# Get the image and text features
with torch.no_grad():
    image_features = model.encode_image(img_input)
    text_features = model.encode_text(text_inputs)

# Normalize the features
image_features /= image_features.norm(p=2, dim=-1, keepdim=True)
text_features /= text_features.norm(p=2, dim=-1, keepdim=True)

# Calculate cosine similarity
similarity = (image_features @ text_features.T).squeeze(0)
similarity = similarity.cpu().numpy()

# Print results
for i, text in enumerate(texts):
    print(f"Similarity to '{text}': {similarity[i]:.4f}")


Similarity to 'A dog': 0.1658
Similarity to 'A cat': 0.2170
Similarity to 'A person': 0.1683
Similarity to 'A golden retriever': 0.1763
Similarity to 'An orange Tabby Cat sitting': 0.2661
Similarity to 'car': 0.1598
